In [ ]:
from pathlib import Path
from collections import defaultdict
import torch


In [ ]:
pcmap = Path('../../data/rcv1v2/pregenerated/parent_child_map.txt').open().readlines()
# pcmap = Path('../../data/nyt/pregenerated/parent_child_map.txt').open().readlines()
# pcmap = Path('../../data/EURLEX57K/pregenerated/parent_child_map_treecode.txt').open().readlines()
pcmap = [line.strip().split('\t') for line in pcmap]

taxonomy = defaultdict(list)
for parent, child in pcmap:
    if parent == 'root':
        parent = 'Root'
    taxonomy[parent].append(child)
    
taxonomy = dict(taxonomy)

In [ ]:
tx = Path('rcv1.taxonomy').open('w')
# tx = Path('nyt.taxonomy').open('w')
# tx = Path('EURLEX57K.taxonomy').open('w')
start = 'Root'
def add_taxonomy(parent):
    if parent in taxonomy:
        tx.write(f'{parent}\t')
        tx.write('\t'.join(taxonomy[parent])+'\n')
        return [add_taxonomy(child) for child in taxonomy[parent]]
add_taxonomy(start)
tx.close()

In [ ]:
label_dict = {}
hiera = defaultdict(set)
with open('rcv1.taxonomy', 'r') as f:
    label_dict['Root'] = -1
    for line in f.readlines():
        line = line.strip().split('\t')
        for i in line[1:]:
            if i not in label_dict:
                label_dict[i] = len(label_dict) - 1
            hiera[label_dict[line[0]]].add(label_dict[i])
    label_dict.pop('Root')
    hiera.pop(-1)
value_dict = {i: v for v, i in label_dict.items()}
torch.save(value_dict, '../../data/rcv1v2/pregenerated/value_dict.pt')
torch.save(hiera, '../../data/rcv1v2/pregenerated/slot.pt')

In [ ]:
label_dict = {}
hiera = defaultdict(set)
with open('nyt.taxonomy', 'r') as f:
    label_dict['Root'] = -1
    for line in f.readlines():
        line = line.strip().split('\t')
        for i in line[1:]:
            if i not in label_dict:
                label_dict[i] = len(label_dict) - 1
            hiera[label_dict[line[0]]].add(label_dict[i])
    label_dict.pop('Root')
    hiera.pop(-1)
value_dict = {i: v for v, i in label_dict.items()}
torch.save(value_dict, '../../data/nyt/pregenerated/value_dict.pt')
torch.save(hiera, '../../data/nyt/pregenerated/slot.pt')

In [ ]:
label_set = set()
hiera = defaultdict(set)
with open('EURLEX57K.taxonomy', 'r') as f:
    label_set.add('Root')
    for line in f.readlines():
        line = line.strip().split('\t')
        parent, children = line[0], line[1:]
        for child in children:
            if child not in label_set:
                label_set.add(child)
            hiera[parent].add(child)
    label_set.remove('Root')
    hiera.pop('Root')
print(label_set)
print(hiera)

value_dict = {i:v for i,v in enumerate(label_set)}
label2idx = {v:i for i,v in value_dict.items()}
hiera = {label2idx[k]:[label2idx[label] for label in v] for k,v in hiera.items()}
# print(len(value_dict))
torch.save(value_dict, '../../data/EURLEX57K/pregenerated/value_dict.pt')
torch.save(hiera, '../../data/EURLEX57K/pregenerated/slot.pt')